In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from AnimalShelter import AnimalShelter

import base64 #had to import for image to work




###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Mypassword"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # load image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Brian Golden'))),
    html.Hr(),
    html.Div(
        
    dcc.RadioItems( # filter options! as radio buttons too!
        id='filter-type',
        options=[
            {'label':'Water Rescue','value':'water'},
            {'label':'Mountain Rescue','value':'mountain'},
            {'label':'Disaster Rescue','value':'diaster'},
            {'label':'Reset','value':'reset'}
        ],
        value='reset'
    )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action='native',
        page_current = 0,
        page_size = 10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
def getMarker(dff,i): # utility method to make map markers for adding more than one to the map
    return dl.Marker(position=[dff.iloc[i,13],dff.iloc[i,14]], children=[
                dl.Tooltip(dff.iloc[i,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[i,9])
                ])
            ])


    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# below runs the queries for each filter type
    
    cur = None
    if filter_type == 'water':
        cur = shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix",
        "Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})
        
    elif filter_type == 'mountain':
        cur = shelter.read({"animal_type":"Dog","breed":{"$in":["German Shephard","Alaskan Malamute",
        "Old English Sheepdog","Siberian Husky","Rottweiler"]},"sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})
    
    elif filter_type == 'diaster':
        cur = shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd",
        "Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})
    else:
        cur = shelter.read({}) # if reset we just get the whole list
        
    df = pd.DataFrame.from_records(cur)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('filter-type', 'value')])
def update_graphs(viewData,filter_type):
    breedTypes = [] # this holds the breed names
    breedValues = []# this holds the values of each breed type
    otherCounter = 0 # for the unfiltered view we needed to tone down how much was showing. This is the counter
    # honestly the below if statements are not exactly pretty but it works
    # I figured it would be better to have values show correctly and this was the best way I could think of
    if filter_type == 'water': 
        breedTypes = ["Labrador Retriever Mix",
        "Chesapeake Bay Retriever","Newfoundland"] # add water breeds to the name list
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Labrador Retriever Mix", "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Chesapeake Bay Retriever", "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Newfoundland", "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}).count())
        # above appends the value of each breed that meets the criteria
    elif filter_type == 'diaster': # this does the same thing as water
        breedTypes = ["Doberman Pinscher","German Shepherd",
        "Golden Retriever","Bloodhound","Rottweiler"]
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Doberman Pinscher","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"German Shepherd","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Golden Retriever","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Bloodhound","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Rottweiler","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}).count())
    elif filter_type == 'mountain': # same thing again
        breedTypes = ["German Shephard","Alaskan Malamute",
        "Old English Sheepdog","Siberian Husky","Rottweiler"]
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"German Shephard","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Alaskan Malamute","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Old English Sheepdog","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Siberian Husky","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}).count())
        breedValues.append(shelter.read({"animal_type":"Dog","breed":"Rottweiler","sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}).count())
        
    else: # for unfiltered view I wanted to make it cool
        tempBreed = shelter.getDistinct("breed") # I made a method in the pymongo class to get all Distinct values
        breedTypes.append('Other Breeds') # add the other breeds to start
        for x in tempBreed: # run through each breed value in the master list
            cur = shelter.read({'breed':x}) # get the cursor of each breed
            if cur.count() > 50: # if more than 50 of that breed, add to the master list
                breedTypes.append(x)
            else:
                otherCounter += cur.count() #otherwise add that count to other
        breedValues.append(otherCounter) # append other count first to match other breeds
        for x in breedTypes: #loop through all breeds not other, and add their values to the list
            if x != 'Other Breeds':
                cur = shelter.read({'breed':x})
                breedValues.append(cur.count())
    
    dff = pd.DataFrame.from_dict(viewData) #we dont need the data from the dataframe for this chart
    return [
        dcc.Graph(            
            figure = px.pie(dff,values=breedValues,names=breedTypes,hole=0.3)# make our cute pie chart
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('filter-type', 'value')])
def update_map(viewData,filter_type):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    markers = [dl.TileLayer(id="base-layer-id")] # start with this and append markers
    if filter_type == 'diaster' or filter_type == 'mountain':
        for i in range(0,4): #in a dynamic database this wouldnt be hard coded but I couldnt figure out how to get the row count easily
            markers.append(getMarker(dff,i)) #to do multiple markers
    else: 
        for i in range(0,10):
            markers.append(getMarker(dff,i))
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.5,-97.48], zoom=10, children=markers)
    ]



app